In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit, KFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import pandas as pd
import random
from collections import defaultdict
from assemble_data import *
from feature_extraction import *
import os
import time
from tqdm import tqdm

path = os.getcwd()
if path.endswith('Modeling'):
    os.chdir('../APS_Data')
    path = os.getcwd()

accelerations = ['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz']    # Features list for creating string titles
freq_funcs = ['MNF', 'MDF', 'PKF', 'MNP', 'TTP', 'FR', 'PSR', 'FMD', 'FMN']
time_funcs = ['AAV', 'MAV', 'MMAV', 'SSI', 'VAR', 'ZC', 'SSC', 'WL', 'AAC', 'WAMP']

In [21]:

def extract_expanding_features(features, accel, misc):
    cols = misc
    cols.append('Trial')   
    cols.append('MS Score')

    for f in features: 
        for a in accel:
            cols.append(a + '_' + f)   
    table = pd.read_csv('Feature_Tables/features.csv', usecols=cols)  # concatenating aceeleration names and feature names
    misc_table = pd.read_csv('Feature_Tables/features.csv', usecols=misc + ['Trial'])
    
    X = []
    Y = []
    trials = table['Trial'].unique()

    for t in trials:
        misc_data = misc_table[(misc_table['Trial'] == t)]  # categorical features
        data = table[(table['Trial'] == t)]
        y = []              
        for v in data['MS Score'].values:               
            # y.append(0 if v < 3 else 1 if v < 7 else 2)     # ternary class             
            y.append(0 if v < 3 else 1)     # binary class              
        Y.append(y)             
        # creating label vector
        data.drop(columns=['Trial', 'MS Score'], inplace=True)
        misc_data.drop(columns=['Trial'], inplace=True)

        cumulative_data = data.cumsum(axis=0)
        cumulative_average_data = cumulative_data.divide(range(1, len(cumulative_data) + 1), axis=0)
        subject_code = np.zeros(35)
        subject_code[int(crop_name(t)[:2]) - 1] = 1
        extended_code = np.tile(subject_code, (data.shape[0], 1))

        X.append(np.concatenate((cumulative_average_data.values, data.values, extended_code), axis=1))
        # X.append(np.concatenate((data.values, extended_code), axis=1)) # for current window
    return np.array(X), np.array(Y)


def extract_time_offset_features(features, accel, misc, n=1):
    cols = []
    cols.append('Trial')
    cols.append('MS Score')

    for f in features:
        for a in accel:
            cols.append(a + '_' + f)
    # concatenating aceeleration names and feature names

    data_table = pd.read_csv('Feature_Tables/features.csv', usecols=cols)
    baseline_table = pd.read_csv('Feature_Tables/features_baseline.csv', usecols=cols)
    misc_table = pd.read_csv('Feature_Tables/features.csv', usecols=misc + ['Trial'])
    base_misc_table = pd.read_csv('Feature_Tables/features_baseline.csv', usecols=misc + ['Trial'])
    # read data into pandas DF's
    X = []
    Y = []
    trials = data_table['Trial'].unique()

    for t in trials:
        base = baseline_table[baseline_table['Trial'] == t]     # baseline data
        base_misc_data = base_misc_table[(base_misc_table['Trial'] == t)]
        data = data_table[(data_table['Trial'] == t)]   # IMU data
        data = pd.concat([base, data], axis=0)
        misc_data = misc_table[(misc_table['Trial'] == t)]  # categorical features
        misc_data = pd.concat([base_misc_data, misc_data], axis=0)
        # indexing data tables by trial
        y = []
        for v in data['MS Score'].values:
            # y.append(0 if v < 2 else 1 if v < 5 else 2)     # ternary class
            y.append(0 if v < 3 else 1)     # binary class
        Y.append(y)
        # creating label vector
        base.drop(columns=['Trial', 'MS Score'], inplace=True)
        data.drop(columns=['Trial', 'MS Score'], inplace=True)
        misc_data.drop(columns=['Trial'], inplace=True) # remove unnecessary data columns

        subject_code = np.zeros(35)
        subject_code[int(crop_name(t)[:2]) - 1] = 1
        extended_code = np.tile(subject_code, (data.shape[0], 1))
        # create one hot encoding for subject ID

        offset_data = []    # to store each offset before concatenating them all to the current window
        for i in range(n):  # loop through n previous windows
            length = min(len(data), i + 1)  # account for short trials with fewer windows than n
            offset = np.repeat(base.values, length, axis=0)     # baseline values for all the windows that don't have enough previous data 
            cropped_data = data.values[:(-1 * (i + 1))]     # select previous data in lower diagonal
            offset = np.concatenate((offset, cropped_data), axis=0)
            offset_data.append(offset)
        # this loop iterates through the n=i previous data for an entire trial 
        # is does NOT loop through each window adding their respective offsets
        offset_data.append(data.values)
        offset_data.append(misc_data.values)
        offset_data.append(extended_code)
        # concatenate previous vectors, current vector, and misc vector
        X.append(np.concatenate(offset_data, axis=1))
    return np.array(X), np.array(Y)

In [37]:
misc = ['Susceptibility', 'Gender', 'Age', 'APS', 'Task', 'Sleep_hours', 'Normal_sleep', 'Other_naps', 'Stress_level', 'Fatigue_level', 'Amount_of_exercies', 'Eat_in_12', 'Last_eat_time']
accel = [a[:2] for a in accelerations]
features = ['MNF', 'MDF', 'PKF', 'MNP', 'TTP', 'FR', 'PSR', 'FMN', 'FMD', 'AAV', 'MAV', 'MMAV', 'SSI', 'VAR', 'ZC', 'SSC', 'WL', 'AAC', 'WAMP']
# select which misc data, acceleration type, and features to extract

X, Y = extract_time_offset_features(features=features, accel=accel, misc=misc, n=3)

/var/folders/9p/n09d6qfj5d99hs5gvj02l5kr0000gn/T/ipykernel_19844/632829838.py:98: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(X), np.array(Y)


In [38]:
feat_table = pd.read_csv('feature_tables/features.csv')

def custom_kfold(k):
    trial_names = feat_table['Trial'].unique()
    abs_index = {}
    group_count = defaultdict(int)
    for count, t in enumerate(trial_names):
        abbr = t[:10]
        if group_count[abbr] == 0:
            abs_index[abbr] = count
        group_count[abbr] += 1
    
    group_names = list(group_count.keys())
    random.shuffle(group_names)

    fold = 0
    fold_mat = []
    splits = []
    for i in range(k):
        fold_mat.append(list())

    for n in group_names:
        for j in range(group_count[n]):
            fold_mat[fold].append(abs_index[n] + j)
            fold += 1
            fold %= k

    for i in range(k):
        test = fold_mat[i]
        train = []
        for j in range(k):
            if j != i:
                train += fold_mat[j]
        splits.append((train, test))

    return splits

def cross_validation(X, Y, k=5, reps=50, trees=100, loss='gini', calc_auc=False):
    kf = custom_kfold(k)
    super_sum = 0
    f_ones = 0
    auc_scores = 0
    repetitions = reps
    for i in range(repetitions): # simulate many times to find the true average
        running_confusion = []
        for train, test in kf:
            auc = 0
            try:
                X_train = np.concatenate(X[train], axis=0)
                X_train = np.vstack(X_train)
                X_test = np.concatenate(X[test], axis=0)
                X_test = np.vstack(X_test)
                Y_train = np.concatenate(Y[train], axis=0)
                Y_train = np.vstack(Y_train)
                Y_test = np.concatenate(Y[test], axis=0)
                Y_test = np.vstack(Y_test)
            except:
                print('Error. Continuing.')
                continue
            # index data by train/test split and concatenate the samples of all the trials
            pipe = Pipeline([('rf', RandomForestClassifier(criterion=loss, n_estimators=trees, random_state=42))])    # define model
            pipe.fit(X_train, Y_train.ravel())  # train model
            pred = pipe.predict(X_test) # predict on test set
            if calc_auc:
                probas = pipe.predict_proba(X_test) 
            score = pipe.score(X_test, Y_test)
            f1 = f1_score(pred, Y_test, average='macro')
            cm = confusion_matrix(Y_test, pred)
            if calc_auc:
                auc = roc_auc_score(Y_test.ravel(), probas[:, 1], multi_class='ovr', average='macro')
            # if len(cm) == 2:  # to account for ternary confusion matrices that have 0 predictions in a class
            #     cm = np.pad(cm, ((0,1), (0,1)), mode='constant')  
            # running_confusion.append(cm)
            super_sum += score
            f_ones += f1
            auc_scores += auc
        # confusion = sum(running_confusion)
        # if i == 0:  # print confusion matrix for the first simulation
        #     plt.imshow(confusion, cmap='Blues')
        #     for i in range(confusion.shape[0]):
        #         for j in range(confusion.shape[1]):
        #             plt.text(j, i, confusion[i, j], ha='center', va='center', color='black')
        #     plt.xticks(np.arange(confusion.shape[1]))
        #     plt.yticks(np.arange(confusion.shape[0]))

    accuracy = super_sum / (k * repetitions)
    f_one_score =  f_ones / (k * repetitions)
    area_under_curve = auc_scores / (k * repetitions)
    return accuracy, f_one_score, area_under_curve

In [ ]:
results = cross_validation(X, Y, 5, 200, trees=50, loss='entropy', calc_auc=True)
print([round(x, 4) for x in results])

# Hyperparameter Search

In [ ]:
table = pd.read_csv('Feature_Tables/features.csv')
misc = ['Susceptibility', 'Gender', 'Age', 'APS', 'Task', 'Sleep_hours', 'Normal_sleep', 'Other_naps', 'Stress_level', 'Fatigue_level', 'Amount_of_exercies', 'Eat_in_12', 'Last_eat_time']
table.drop(columns=['Time', 'Trial', 'MS Score'], inplace=True)
table.drop(columns=misc, inplace=True)
feat = table.columns
print(len(feat))
feature_names = ['1 - ' + x for x in feat]
# create list of feature names in string form

feature_names.extend(['2 - ' + x for x in feat])
feature_names.extend(['3 - ' + x for x in feat])
feature_names.extend(['4 - ' + x for x in feat])
feature_names.extend(['5 - ' + x for x in feat])
feature_names.extend(['6 - ' + x for x in feat])
feature_names.extend(['7 - ' + x for x in feat])
feature_names.extend(['8 - ' + x for x in feat])
feature_names.extend(['9 - ' + x for x in feat])
feature_names.extend(['10 - ' + x for x in feat])
feature_names.extend(['0 - ' + x for x in feat])
feature_names.extend(misc)
feature_names.extend(['Subject Code' for i in range(35)])
print(feature_names[55])
# crude, but creates feature names for offset windows

X_train = np.concatenate(X, axis=0)
X_train = np.vstack(X_train)
y_train = np.concatenate(Y, axis=0)
y_train = np.vstack(y_train)
# train with all the data

forest = RandomForestClassifier(n_estimators=400, random_state=0)   # define model
forest.fit(X_train, y_train.ravel())    # train model

start_time = time.time()    # timer
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
elapsed_time = time.time() - start_time
print(len(importances))
print(len(feature_names))
feature_importances = pd.Series(importances, index=feature_names)
# calculate feature importances

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")
for f, i in zip(feature_names, feature_importances):
    print(f)
for f, i in zip(feature_names, feature_importances):
    print(100 * i)

In [ ]:
misc = ['Susceptibility', 'Gender', 'Age', 'APS', 'Task', 'Sleep_hours', 'Normal_sleep', 'Other_naps', 'Stress_level', 'Fatigue_level', 'Amount_of_exercies', 'Eat_in_12', 'Last_eat_time']

misc.extend(features)
misc.append('Random')
for i in range(11):
    indices = [index for index, string in enumerate(feature_names) if string[:2] == str(i) + ' ']
    total = sum(feature_importances[i] for i in indices)
    print(i)

for i in range(11):
    indices = [index for index, string in enumerate(feature_names) if string[:2] == str(i) + ' ']
    total = sum(feature_importances[i] for i in indices)
    print(round(100 * total, 5))
# calculate importance sums for acceleration types and individual features

In [39]:
def hp_grid_search(X, Y, tree=[50, 300]):
    criterion = ['gini', 'entropy']   # create list on 10^x scale
    tree_range = np.linspace(tree[0], tree[1], 6)   # create list on 10^x scale
    best_score = 0
    best_params = [0, 0]

    for t in tqdm(tree_range):
        for c in criterion:
            score = cross_validation(X, Y, 5, 100, int(t), c)[0]    # accuracy score
            if score > best_score:
                # update best
                best_score = score
                best_params[0] = t
                best_params[1] = c
    return best_params, best_score
    # [C, gamma]

params, accuracy = hp_grid_search(X, Y)
print("Best parameters")
print("Num Estimators:", params[0])
print("Loss Function:", params[1])
print('Score:', accuracy)

100%|██████████| 6/6 [2:08:32<00:00, 1285.40s/it]

Best parameters
Num Estimators: 50.0
Loss Function: entropy
Score: 0.7831160305073366
